## Churn Prediction with Complete Dataset(12GB)
In this notebook we will upload the complete datalogs(12GB) from S3 and based on our analysis with shorter dataset in Sparkify_Project.ipynb, we will replicate all the below steps on the complete data.
1. Data Understanding and Data Cleaning
2. Exploratory Data Analysis
3. Feature Engineering
4. Modelling  & Evaluation
5. Results and Summary

In [86]:
# import libraries
from pyspark.sql import functions as F
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, split, udf, countDistinct, mean, to_date
from pyspark.sql.functions import sum,round
from pyspark.sql.functions import min
from pyspark.sql.functions import max
from pyspark.sql.functions import count
from pyspark.sql.functions import isnan, when, count, col
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import udf
import re
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType, DoubleType
import datetime
from pyspark.ml import Pipeline
from pyspark.ml.feature import StandardScaler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator, Evaluator
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.mllib.evaluation import MulticlassMetrics
import numpy as np

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
# import libraries
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**1. Data Understanding and Data Cleaning :**

In [66]:
# Starter code
from pyspark.sql import SparkSession

# Create spark session
spark = SparkSession \
    .builder \
    .appName("Sparkify") \
    .getOrCreate()

# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"
df = spark.read.json(event_data)
df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [67]:
# Shape of dataframe
print("Shape of dataframe : ",(df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of dataframe :  (26259199, 18)

In [87]:
# Schema of dataframe
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [69]:
# Exploring duplicate rows

# Total number of rows in Dataframe
print("Dataframe total row count: ",df.count())

# Total distinct number of rows
print("Distinct rows count: ",df.dropDuplicates().count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Dataframe total row count:  26259199
Distinct rows count:  26259199

Observation : There are no duplicate rows.

In [70]:
# Explore nan values
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|artist|auth|firstName|gender|itemInSession|lastName|length|level|location|method|page|registration|sessionId|song|status| ts|userAgent|userId|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+
|     0|   0|        0|     0|            0|       0|     0|    0|       0|     0|   0|           0|        0|   0|     0|  0|        0|     0|
+------+----+---------+------+-------------+--------+------+-----+--------+------+----+------------+---------+----+------+---+---------+------+

In [71]:
# Explore null values
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
| artist|auth|firstName|gender|itemInSession|lastName| length|level|location|method|page|registration|sessionId|   song|status| ts|userAgent|userId|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
|5408927|   0|   778479|778479|            0|  778479|5408927|    0|  778479|     0|   0|      778479|        0|5408927|     0|  0|   778479|     0|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+

In [73]:
# Look at Userid where gender column has missing values
df1 = df.filter(df["gender"].isNull())
df1.groupBy("userId").pivot("auth").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+----------+
| userId|Guest|Logged Out|
+-------+-----+----------+
|1261737| 4194|    774285|
+-------+-----+----------+

In [72]:
# Count number of rows for  userId = 1261737
df.filter(df["userId"]=="1261737").count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

778479

Observation : Logs corresponding to userId 1261737 represents missing values in gender, Name,location,registration,useragent columns. Since these are either Guest or Logged Out users , therfore we can remove these 778479 rows(3%) from our data

In [88]:
# Remove rows with userID"1261737"
df = df.filter('userID != "1261737" ')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [89]:
# Explore null values again
df.select([count(when(col(c).isNull(), c)).alias(c) for c in df.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
| artist|auth|firstName|gender|itemInSession|lastName| length|level|location|method|page|registration|sessionId|   song|status| ts|userAgent|userId|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+
|4630448|   0|        0|     0|            0|       0|4630448|    0|       0|     0|   0|           0|        0|4630448|     0|  0|        0|     0|
+-------+----+---------+------+-------------+--------+-------+-----+--------+------+----+------------+---------+-------+------+---+---------+------+

In [75]:
# Number of distinct page values
print("Number of distinct page values")
df.select(countDistinct("page")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of distinct page values
+--------------------+
|count(DISTINCT page)|
+--------------------+
|                  22|
+--------------------+

In [76]:
# number of records per page type
page_records = df.groupby("page").count()
page_records.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+--------+
|                page|   count|
+--------------------+--------+
|              Cancel|    5003|
|    Submit Downgrade|    6494|
|         Thumbs Down|  239212|
|                Home| 1343102|
|           Downgrade|  184240|
|         Roll Advert|  385212|
|              Logout|  296005|
|       Save Settings|   29516|
|Cancellation Conf...|    5003|
|               About|   92759|
| Submit Registration|     401|
|            Settings|  147074|
|               Login|  296350|
|            Register|     802|
|     Add to Playlist|  597921|
|          Add Friend|  381664|
|            NextSong|20850272|
|           Thumbs Up| 1151465|
|                Help|  155100|
|             Upgrade|   50507|
+--------------------+--------+
only showing top 20 rows

In [77]:
# Number Of records per auth type
df.groupby("auth").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+--------+
|      auth|   count|
+----------+--------+
|Logged Out|  774285|
| Cancelled|    5003|
|     Guest|    4194|
| Logged In|25475717|
+----------+--------+

In [78]:
# Number of unique songs
print ("Number of unique songs :")
df.select(countDistinct("song")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of unique songs :
+--------------------+
|count(DISTINCT song)|
+--------------------+
|              253564|
+--------------------+

In [79]:
# Number of unique Users
print("Number of unique Users:")
df.select(countDistinct("userId")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of unique Users:
+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                 22278|
+----------------------+

In [22]:
# Number of unique artists
print("Number of unique artists:")
df.select(countDistinct("artist")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of unique artists:
+----------------------+
|count(DISTINCT artist)|
+----------------------+
|                 38337|
+----------------------+

In [23]:
# Number Of records per level type#
df.groupby("level").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+
|level|   count|
+-----+--------+
| free| 5408240|
| paid|20072480|
+-----+--------+

In [80]:
# New Shape of dataframe
print("Shape of dataframe : ",(df.count(), len(df.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of dataframe :  (26259199, 18)

In [90]:
# Adding columns for datetime and date from the timestamp columns(ts and registration)

# Function to get datetime
get_time= udf(lambda x: datetime.datetime.fromtimestamp(x / 1000.0).strftime("%Y-%m-%d %H:%M:%S"))

# Creating a data frame with addional columns for datetime
data = df.withColumn("datetime", get_time(df.ts))
data = data.withColumn("reg_datetime", get_time(df.registration))

spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")

# Adding date columns
data = data.withColumn("date", to_date(col("datetime"),"yyyy-MM-dd"))
data = data.withColumn("reg_date", to_date(col("reg_datetime"),"yyyy-MM-dd"))

# Look at the original timestamp and added columns
data.select("ts","datetime","date","registration","reg_datetime","reg_date").show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------------------+----------+-------------+-------------------+----------+
|           ts|           datetime|      date| registration|       reg_datetime|  reg_date|
+-------------+-------------------+----------+-------------+-------------------+----------+
|1538352001000|2018-10-01 00:00:01|2018-10-01|1533734541000|2018-08-08 13:22:21|2018-08-08|
|1538352002000|2018-10-01 00:00:02|2018-10-01|1537500318000|2018-09-21 03:25:18|2018-09-21|
|1538352002000|2018-10-01 00:00:02|2018-10-01|1536414505000|2018-09-08 13:48:25|2018-09-08|
|1538352003000|2018-10-01 00:00:03|2018-10-01|1534386660000|2018-08-16 02:31:00|2018-08-16|
|1538352003000|2018-10-01 00:00:03|2018-10-01|1537381415000|2018-09-19 18:23:35|2018-09-19|
|1538352004000|2018-10-01 00:00:04|2018-10-01|1537602560000|2018-09-22 07:49:20|2018-09-22|
|1538352005000|2018-10-01 00:00:05|2018-10-01|1536563853000|2018-09-10 07:17:33|2018-09-10|
|1538352006000|2018-10-01 00:00:06|2018-10-01|1538069376000|2018-09-27 17:29:36|

In [82]:
# Minimum and Maximum log time in data
data.select("datetime").agg(min("datetime"),max("datetime")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|      min(datetime)|      max(datetime)|
+-------------------+-------------------+
|2018-10-01 00:00:01|2018-12-01 00:00:02|
+-------------------+-------------------+

%md
Observation : Logs data is available for 2 months from 1st Oct,2018 to 1st Dec,2018

In [27]:
# Minimum and Maximum user registration time in data
data.select("reg_datetime").agg(min("reg_datetime"),max("reg_datetime")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|  min(reg_datetime)|  max(reg_datetime)|
+-------------------+-------------------+
|2017-10-14 22:05:25|2018-12-03 07:23:42|
+-------------------+-------------------+

Observation : Just based on available logs data, on an average, life of Non Churn user are more than Churn Users. 

More observations from data exploration: 
- After every "cancellation" event page there is always a "cancellation confirmation" event page
- After "downgrade" event, there is no changed observed for user for level but after every "submit downgrade" event, level changes from paid to free.

**2. Exploratory Data Analysis:**

In [91]:
#Adding a label column representing Churn event
Is_Churn = udf(lambda x : int(1) if x=="Cancellation Confirmation" else int(0))
data = data.withColumn("Is_Churn", Is_Churn(data.page))

data.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+---------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+-------------------+-------------------+----------+----------+--------+
|              artist|     auth|firstName|gender|itemInSession|lastName|    length|level|            location|method|    page| registration|sessionId|                song|status|           ts|           userAgent| userId|           datetime|       reg_datetime|      date|  reg_date|Is_Churn|
+--------------------+---------+---------+------+-------------+--------+----------+-----+--------------------+------+--------+-------------+---------+--------------------+------+-------------+--------------------+-------+-------------------+-------------------+----------+----------+--------+
|           Popol Vuh|Logged In|    Shlok|     M|          278| Johnson| 524.32934| paid|Dallas-Fort Worth...|   PUT|Next

In [85]:
# Number of total unique Users
print("Total number of unique Users:")
data.select(countDistinct("userId")).show()

# Number of unique Users
print("Number of unique Churned Users:")
data.filter(data["Is_Churn"]==1).select(countDistinct("userId")).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Total number of unique Users:
+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                 22278|
+----------------------+

Number of unique Churned Users:
+----------------------+
|count(DISTINCT userId)|
+----------------------+
|                  5003|
+----------------------+

In [30]:
# Distribution of Is_Churn amoung auth catergories
data.groupby("Is_Churn").pivot("auth").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+---------+
|Is_Churn|Cancelled|Logged In|
+--------+---------+---------+
|       0|     null| 25475717|
|       1|     5003|     null|
+--------+---------+---------+

In [31]:
# Distribution of Is_Churn amoung gender catergories
data.groupby("Is_Churn").pivot("gender").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------+--------+
|Is_Churn|       F|       M|
+--------+--------+--------+
|       0|12178811|13296906|
|       1|    2347|    2656|
+--------+--------+--------+

In [32]:
# Distribution of Is_Churn amoung level catergories
data.groupby("Is_Churn").pivot("level").count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+-------+--------+
|Is_Churn|   free|    paid|
+--------+-------+--------+
|       0|5406661|20069056|
|       1|   1579|    3424|
+--------+-------+--------+

Observation :Free users are two times more likely to churn compared to Non-Churn users

In [33]:
# Average number of songs listened by Churn and Non Churn Users during the available logs data
from pyspark.sql.functions import count
Number_of_songs_per_user = data.groupBy("userId").agg(count("song").alias("Number_of_songs_listened"),max("Is_Churn").alias("Is_Churn"))
dataframe1 = Number_of_songs_per_user
dataframe1.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------------+--------+
| userId|Number_of_songs_listened|Is_Churn|
+-------+------------------------+--------+
|1000280|                    1022|       1|
|1002185|                    1778|       0|
|1017805|                     250|       0|
|1030587|                    1472|       0|
|1033297|                     236|       0|
|1057724|                    3847|       0|
|1059049|                     559|       0|
|1069552|                     455|       0|
|1071308|                    1409|       1|
|1076191|                      47|       1|
+-------+------------------------+--------+
only showing top 10 rows

In [34]:
#SessionID_exploration

# Average Numbers of sessions with Churn and Non Churn Users during the available logs data
SessionID_exploration = data.groupBy("userID").agg(count("sessionId").alias("Number_of_Sessions"),sum("Is_Churn").alias("Is_Churn"))
dataframe2 = SessionID_exploration
dataframe2.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+--------+
| userID|Number_of_Sessions|Is_Churn|
+-------+------------------+--------+
|1200956|              4143|     0.0|
|1396135|              1144|     0.0|
|1718034|               875|     1.0|
|1567623|              1375|     0.0|
|1396828|              1519|     1.0|
|1633577|              4134|     0.0|
|1347224|               860|     0.0|
|1102913|               904|     0.0|
|1083324|              1784|     0.0|
|1190352|              2622|     0.0|
+-------+------------------+--------+
only showing top 10 rows

In [35]:
# Average Session length with Churn and Non Churn Users during the available logs data
session_data = data.groupBy("userID","sessionId").agg(max("Is_Churn").alias("Is_churn"),max("datetime").alias("max_time"),min("datetime").alias("min_time"))
session_data =session_data.withColumn("Session_length",(unix_timestamp("max_time")-unix_timestamp("min_time")) )
dataframe3 = session_data.groupBy("userID").agg(max("Is_churn").alias("Is_churn"),mean("Session_length").alias("Avg_session_length_seconds"))
dataframe3.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+--------------------------+
| userID|Is_churn|Avg_session_length_seconds|
+-------+--------+--------------------------+
|1000280|       1|        11694.363636363636|
|1002185|       0|        26084.352941176472|
|1017805|       0|        21136.666666666668|
|1030587|       0|         32943.63636363636|
|1033297|       0|                   11821.0|
|1057724|       0|                 23844.775|
|1059049|       0|                   28119.6|
|1069552|       0|         9298.416666666666|
|1071308|       1|        19334.944444444445|
|1076191|       1|         5188.333333333333|
+-------+--------+--------------------------+
only showing top 10 rows

In [36]:
# Users age from the registration
from pyspark.sql.functions import col, max
from pyspark.sql.functions import datediff,col
user_age = data.groupBy("userId").agg(max("Is_Churn").alias("Is_Churn"),min("reg_date").alias("reg_time"),max("date").alias("max_time"))
dataframe4 = user_age.withColumn("User_age_days_from_registration", datediff(col("max_time"),col("reg_time")).alias("user_age_days"))
dataframe4.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+----------+----------+-------------------------------+
| userId|Is_Churn|  reg_time|  max_time|User_age_days_from_registration|
+-------+--------+----------+----------+-------------------------------+
|1000280|       1|2018-08-28|2018-11-13|                             77|
|1002185|       0|2018-09-24|2018-11-28|                             65|
|1017805|       0|2018-09-02|2018-10-27|                             55|
|1030587|       0|2018-07-21|2018-11-29|                            131|
|1033297|       0|2018-08-03|2018-11-27|                            116|
|1057724|       0|2018-08-25|2018-11-29|                             96|
|1059049|       0|2018-07-11|2018-11-22|                            134|
|1069552|       0|2018-07-25|2018-11-28|                            126|
|1071308|       1|2018-09-07|2018-11-10|                             64|
|1076191|       1|2018-09-18|2018-10-17|                             29|
+-------+--------+----------+----------+-----------

In [37]:
# Users life within our available logs timing data
from pyspark.sql.functions import col, max
from pyspark.sql.functions import datediff,col
user_life = data.groupBy("userId").agg(max("Is_Churn").alias("Is_Churn"),min("date").alias("min_time"),max("date").alias("max_time"))
dataframe5 = user_life.withColumn("User_life_days_after_1st_October2018", datediff(col("max_time"),col("min_time")).alias("user_life_days"))
dataframe5.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+----------+----------+------------------------------------+
| userId|Is_Churn|  min_time|  max_time|User_life_days_after_1st_October2018|
+-------+--------+----------+----------+------------------------------------+
|1000280|       1|2018-10-01|2018-11-13|                                  43|
|1002185|       0|2018-10-01|2018-11-28|                                  58|
|1017805|       0|2018-10-02|2018-10-27|                                  25|
|1030587|       0|2018-10-11|2018-11-29|                                  49|
|1033297|       0|2018-10-25|2018-11-27|                                  33|
|1057724|       0|2018-10-01|2018-11-29|                                  59|
|1059049|       0|2018-10-16|2018-11-22|                                  37|
|1069552|       0|2018-10-03|2018-11-28|                                  56|
|1071308|       1|2018-10-02|2018-11-10|                                  39|
|1076191|       1|2018-10-09|2018-10-17|                        

***3.Feature Engineering: ***

In [ ]:
# Filtering relevant data for our feature engineering
data_relevant = data.select("userId","ts","datetime","date","registration","reg_datetime","reg_date","sessionId","page","song","length","level","Gender","Is_Churn").sort(col("userId"),col("date").asc())
data_relevant.show()

In [ ]:
# Rename Is_Churn Column
data_relevant = data_relevant.withColumnRenamed("Is_Churn", "Is_Churn(t)")

Adding user's Total Age and user's Available Log time as features:

In [ ]:
#function for converting days to seconds
from pyspark.sql.functions import datediff
days = lambda x: x * 86400

# Given we want to predict 7 days in advance of the Churn event, therefore the defining the users logs data available window
windowval_available = (Window.partitionBy("userId").orderBy(col("ts")/1000).rangeBetween(Window.unboundedPreceding, -days(7)))

max_date = max("date").over(windowval_available)
min_date = min("date").over(windowval_available)
reg_date = min("reg_date").over(windowval_available)

# Addding User age in days as feature
data_relevant = data_relevant.withColumn("age_days(t_7)",(datediff(max_date,reg_date))+1)

# Addding available log time in days as feature                              
data_relevant = data_relevant.withColumn("available_log_days(t_7)",(datediff(max_date,min_date))+1)

Adding Average per day features

In [ ]:
# Add average number of songs played per day until the time the user was last active on app(before 7 day from current time)
data_relevant = data_relevant.withColumn("avg_num_songs_per_day(t_7)", round((count("song").over(windowval_available))/col("available_log_days(t_7)"),4))

# Add Average Length of songs played(music time) per day until the time the user was last active  on app(before 7 day from current time)
data_relevant = data_relevant.withColumn("avg_length_secs_per_day(t_7)", round((sum("length").over(windowval_available))/col("available_log_days(t_7)"),4))

# Add Number of sessions per day until the time the user was last active  on app(before 7 day from current time)
data_relevant = data_relevant.withColumn("avg_number_sessions_per_day(t_7)", round(F.size(F.collect_set("sessionID").over(windowval_available))/col("available_log_days(t_7)"),4))

Adding Negative Features

In [12]:
# Adding fraction submit downgrade activity during available logdata as feature
func_submit_downgrade = udf(lambda page : int(page == 'Submit Downgrade'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_submit_downgrade(t_7)",round((sum(func_submit_downgrade(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ))

# Adding fraction error downgrade activity during available logdata as feature
func_error = udf(lambda page : int(page == 'Error'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_error(t_7)",round((sum(func_error(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction help downgrade activity during available logdata as feature
func_help = udf(lambda page : int(page == 'Help'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_help(t_7)",round((sum(func_help(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Roll Advert activity during available logdata as feature
func_Roll_Advert = udf(lambda page : int(page == 'Roll Advert'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Roll_Advert(t_7)",round((sum(func_Roll_Advert(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Save Settings activity during available logdata as feature
func_Save_Settings = udf(lambda page : int(page == 'Save Settings'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Save_Settings(t_7)",round((sum(func_Save_Settings(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Thumbs Down activity during available logdata as feature
func_Thumbs_Down = udf(lambda page : int(page == 'Thumbs Down'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Thumbs_Down(t_7)",round((sum(func_Thumbs_Down(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Addding Positive features

In [ ]:
# Adding fraction Submit Upgrade activity during available logdata as feature
func_Submit_Upgrade = udf(lambda page : int(page == 'Submit Upgrade'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Submit_Upgrade(t_7)",round((sum(func_Submit_Upgrade(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Add Friend activity during available logdata as feature
func_Add_Friend = udf(lambda page : int(page == 'Add Friend'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Add_Friend(t_7)",round((sum(func_Add_Friend(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Add to Playlist activity during available logdata as feature
func_Add_to_Playlist= udf(lambda page : int(page == 'Add to Playlist'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Add_to_Playlist(t_7)",round((sum(func_Add_to_Playlist(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

# Adding fraction Add to Playlist activity during available logdata as feature
func_Thumbs_Up= udf(lambda page : int(page == 'Thumbs Up'), IntegerType())
data_relevant = data_relevant.withColumn("fraction_Thumbs_Up(t_7)",round((sum(func_Thumbs_Up(col("page"))).over(windowval_available))/(count(col("page")).over(windowval_available)),4 ) )

Adding Is_paid and Is_Male binary features

In [ ]:
# Add Is_paid as a feature
func_paid = udf(lambda level : int(level == 'paid'), IntegerType())
data_relevant = data_relevant.withColumn("Is_paid",func_paid(col("level")))

# Add Is_Male as a feature
func_male = udf(lambda Gender : int(Gender == 'M'), IntegerType())
data_relevant = data_relevant.withColumn("Is_Male",func_male(col("Gender")))

Filtering data for each user containing only latest time data so that for each user we have only one row.

In [15]:
# Ranking data on time so that for every customer only latest time datapoints can be used
data_relevant = data_relevant.withColumn("dense_rank",F.dense_rank().over(Window.partitionBy("userId").orderBy(col("ts").desc())))

# Filtering data at the latest time for every user
data_relevant = data_relevant.where("dense_rank ==1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Removing Users with age less than 7 days

In [ ]:
# Checking Null values in data
data_relevant.select([count(when(col(c).isNull(), c)).alias(c) for c in data_relevant.columns]).show()

In [ ]:
# Distribution of Churn vs Non Churn users among users life less than 7 days
data_relevant.where(data_relevant['age_days(t_7)'].isNull()).groupBy("Is_Churn(t)").agg(count("userId")).show();

In [ ]:
# Removing users with age less than 7 days
data_relevant = data_relevant.filter(data_relevant['age_days(t_7)'].isNotNull())

In [48]:
# Confirming Null values in data
data_relevant.select([count(when(col(c).isNull(), c)).alias(c) for c in data_relevant.columns]).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+---+--------+----+------------+------------+--------+---------+----+-----+------+-----+------+-----------+-------------+-----------------------+--------------------------+----------------------------+--------------------------------+------------------------------+-------------------+------------------+-------------------------+---------------------------+-------------------------+----------------------------+------------------------+-----------------------------+-----------------------+-------+-------+----------+
|userId| ts|datetime|date|registration|reg_datetime|reg_date|sessionId|page| song|length|level|Gender|Is_Churn(t)|age_days(t_7)|available_log_days(t_7)|avg_num_songs_per_day(t_7)|avg_length_secs_per_day(t_7)|avg_number_sessions_per_day(t_7)|fraction_submit_downgrade(t_7)|fraction_error(t_7)|fraction_help(t_7)|fraction_Roll_Advert(t_7)|fraction_Save_Settings(t_7)|fraction_Thumbs_Down(t_7)|fraction_Submit_Upgrade(t_7)|fraction_Add_Friend(t_7)|fraction_Add_to_Playlist(t_7

In [ ]:
# Prepare final featured data
featured_data = data_relevant.drop("userId","ts","datetime","date","registration","reg_datetime","reg_date",\
                                  "sessionId","page","song","length","level","Gender","Is_SubmitDowngrade","available_log_days(t_7)","dense_rank")

In [18]:
# Shape of dataset
print("Shape of dataset : ",(featured_data.count(), len(featured_data.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Shape of dataset :  (20504, 17)

In [ ]:
# Based on the features exploration with small dataset in another notebook,
# we find out that Avg_num_songs_per_day(t_7) & Avg_num_secs_per_day(t_7) are almost completely positively correlated

# Remove Avg_num_secs_per_day(t_7) variable
featured_data = featured_data.drop("avg_length_secs_per_day(t_7)")

In [20]:
featured_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------------+--------------------------+--------------------------------+------------------------------+-------------------+------------------+-------------------------+---------------------------+-------------------------+----------------------------+------------------------+-----------------------------+-----------------------+-------+-------+
|Is_Churn(t)|age_days(t_7)|avg_num_songs_per_day(t_7)|avg_number_sessions_per_day(t_7)|fraction_submit_downgrade(t_7)|fraction_error(t_7)|fraction_help(t_7)|fraction_Roll_Advert(t_7)|fraction_Save_Settings(t_7)|fraction_Thumbs_Down(t_7)|fraction_Submit_Upgrade(t_7)|fraction_Add_Friend(t_7)|fraction_Add_to_Playlist(t_7)|fraction_Thumbs_Up(t_7)|Is_paid|Is_Male|
+-----------+-------------+--------------------------+--------------------------------+------------------------------+-------------------+------------------+-------------------------+---------------------------+-------------------------+----------------------------+----------

In [27]:
# Save the featured data in S3 Storage
featured_data.write.json('s3://aws-emr-resources-134705539818-us-east-2/tranformed_featured_data.json')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
# Load featured data from S3
path = "s3://aws-emr-resources-134705539818-us-east-2/tranformed_featured_data.json"
featured_data = spark.read.json(path)
featured_data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+-------+-------+-------------+--------------------------+--------------------------------+------------------------+-----------------------------+-------------------------+---------------------------+----------------------------+-------------------------+-----------------------+-------------------+------------------+------------------------------+
|Is_Churn(t)|Is_Male|Is_paid|age_days(t_7)|avg_num_songs_per_day(t_7)|avg_number_sessions_per_day(t_7)|fraction_Add_Friend(t_7)|fraction_Add_to_Playlist(t_7)|fraction_Roll_Advert(t_7)|fraction_Save_Settings(t_7)|fraction_Submit_Upgrade(t_7)|fraction_Thumbs_Down(t_7)|fraction_Thumbs_Up(t_7)|fraction_error(t_7)|fraction_help(t_7)|fraction_submit_downgrade(t_7)|
+-----------+-------+-------+-------------+--------------------------+--------------------------------+------------------------+-----------------------------+-------------------------+---------------------------+----------------------------+-------------------------+---------

**4. Modeling and Evaluation**

In [29]:
# date prep for modelling
from pyspark.sql.types import IntegerType
featured_data = featured_data.withColumnRenamed("Is_Churn(t)","label")
featured_data = featured_data.withColumn("label", featured_data["label"].cast(IntegerType()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Train-Test split of dataset**

In [30]:
#Split train/test data
train, test = featured_data.randomSplit([0.8, 0.2], seed=42)

train_size = train.count()
churn_labels_in_train = train.filter(col('label') == 1).count() / train_size
print("train_size :",train_size,", churn_labels_fraction_in_train :",churn_labels_in_train)

test_size = test.count()
churn_labels_in_train =  test.filter(col('label') == 1).count() / test_size
print("test_size :",test_size,", churn_labels_fraction_in_test :",churn_labels_in_train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

train_size : 16437 , churn_labels_fraction_in_train : 0.19133661860436818
test_size : 4062 , churn_labels_fraction_in_test : 0.1930083702609552

In [31]:
# Numberic features list
numeric_columns = ['age_days(t_7)', 'avg_num_songs_per_day(t_7)', 'avg_number_sessions_per_day(t_7)', 'fraction_submit_downgrade(t_7)',\
                   'fraction_error(t_7)', 'fraction_help(t_7)', 'fraction_Roll_Advert(t_7)', 'fraction_Save_Settings(t_7)', 'fraction_Thumbs_Down(t_7)',\
                   'fraction_Submit_Upgrade(t_7)', 'fraction_Add_Friend(t_7)', 'fraction_Add_to_Playlist(t_7)', 'fraction_Thumbs_Up(t_7)']


# Combining multiple numerical features using VectorAssembler
numeric_assembler = VectorAssembler(inputCols = numeric_columns, outputCol = "numericvectorized")

# Standardizing numerical features
scaler = StandardScaler(inputCol = "numericvectorized", outputCol = "numericscaled", withStd = True, withMean = True)

# Adding the two binary features
binary_columns = ['Is_paid', 'Is_Male']
total_assembler = VectorAssembler(inputCols = binary_columns + ["numericscaled"], outputCol = "features")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
#defining fit model for the pipeline
def fit_model(model, paramGrid = None):
  """ build the pipeline, fit the model on train with best crossvalidated model\
  and predict on test data
  
  Args:
    model: model estimator
    
  Returns:
    fitmodel: best fitted model on train data
    results: prediction results on test data
  """
  # define pipeline
  pipeline = Pipeline(stages=[numeric_assembler, scaler, total_assembler, model])

  # cross validate on parameter and fit the best model on train data
  if paramGrid != None:
      crossval = CrossValidator(estimator=pipeline,
                        estimatorParamMaps=paramGrid,
                        evaluator=MulticlassClassificationEvaluator(metricName='f1'),
                        numFolds=3)
      fitmodel = crossval.fit(train)
  else:
      fitmodel = pipeline.fit(train)
  
  #predict on test data
  results = fitmodel.transform(test)

  return fitmodel, results

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [33]:
# defining function for calculating evaluation metrics
def evaluation(results):
  """ calculate accuracy,precision, recall, f1 metrics
  
  Args:
    results: model prediction results on test data
    
  Returns:
    (accuracy,precision, recall, f1) : evaluation metrics
  """

  dataset = results.select(['prediction', 'label'])
  
  tp = dataset.where((dataset.label == 1) & (dataset.prediction == 1)).count()
  fp = dataset.where((dataset.label == 0) & (dataset.prediction == 1)).count()
  tn = dataset.where((dataset.label == 0) & (dataset.prediction == 0)).count()
  fn = dataset.where((dataset.label == 1) & (dataset.prediction == 0)).count()
  total = dataset.count()

  # Add epsilon to prevent division by zero
  accuracy = (tp+tn)/(total)
  precision = tp / (tp + fp + 0.00001)
  recall = tp / (tp + fn + 0.00001)
  f1 = 2 * precision * recall / (precision + recall + 0.00001)

  return(accuracy,precision, recall, f1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

**Logistic Regression with Class weights for imbalance treatment**

In [34]:
# Get weighted column to deal with imbalance of data catergories
balancingRatio = train.filter(col('label') == 1).count() / train.count()
calculateWeights = udf(lambda x: 1 * balancingRatio if x == 0 else (1 * (1.0 - balancingRatio)), DoubleType())
train = train.withColumn("classWeightCol", calculateWeights('label'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [49]:
# Logistic Regression with weights of classes considered

# define model
lr_w = LogisticRegression(maxIter=10,weightCol = 'classWeightCol')

# define parametes grid
lr_w_paramGrid = ParamGridBuilder().addGrid(lr_w.regParam,[0.0, 0.5]).addGrid(lr_w.threshold,[0.5]).build()

# fit model pipeline
lr_w_model, lr_w_results = fit_model(lr_w, paramGrid = lr_w_paramGrid)

# getting best model based on grid search
lr_w_best_model_index = lr_w_model.avgMetrics.index(np.max(lr_w_model.avgMetrics))
lr_w_best_model_param = lr_w_paramGrid[lr_w_best_model_index]
print(lr_w_best_model_param)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='LogisticRegression_bb18f1c271d5', name='regParam', doc='regularization parameter (>= 0).'): 0.5, Param(parent='LogisticRegression_bb18f1c271d5', name='threshold', doc='Threshold in binary classification prediction, in range [0, 1]. If threshold and thresholds are both set, they must match.e.g. if threshold is p, then thresholds must be equal to [1-p, p].'): 0.5}

In [50]:
# Getting best Evaluation metrics
print("LogisticRegression with class weights - accuracy,precision, recall, f1 :", evaluation(lr_w_results))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

LogisticRegression with class weights - accuracy,precision, recall, f1 : (0.758739537173806, 0.41819699150085987, 0.6390306040939974, 0.5055451626438296)

In [51]:
# List of features
list_feats =  binary_columns + numeric_columns

# model features coefficients
lr_w_corr = lr_w_model.bestModel.stages[-1].coefficients

# logistic regression coefficients
print('Logistic regression features importances')
print(list_feats)
print(lr_w_corr)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic regression features importances
['Is_paid', 'Is_Male', 'age_days(t_7)', 'avg_num_songs_per_day(t_7)', 'avg_number_sessions_per_day(t_7)', 'fraction_submit_downgrade(t_7)', 'fraction_error(t_7)', 'fraction_help(t_7)', 'fraction_Roll_Advert(t_7)', 'fraction_Save_Settings(t_7)', 'fraction_Thumbs_Down(t_7)', 'fraction_Submit_Upgrade(t_7)', 'fraction_Add_Friend(t_7)', 'fraction_Add_to_Playlist(t_7)', 'fraction_Thumbs_Up(t_7)']
[0.19542025400328558,0.014413681403436355,-0.11464798577273039,0.12357987427792498,0.1771921088566431,0.0278654605538792,0.0030159609324810193,0.00998071493442744,0.061205246256120796,-0.008600705195865586,0.05771519841430953,0.0219393377255115,-0.009651806008248713,-0.00023186746059301565,-0.08180904441356712]

**Random Forest Modeling**

In [52]:
# Random Forest with no weights of classes considered

# define model
#rf = RandomForestClassifier(weightCol = 'classWeightCol')
rf = RandomForestClassifier()

# define parametes grid
rf_paramGrid = ParamGridBuilder().addGrid(rf.maxDepth,[10,15]).addGrid(rf.numTrees,[100,200]).addGrid(rf.featureSubsetStrategy,["sqrt"]).build()

# fit model pipeline
rfmodel, rfresults = fit_model(rf, paramGrid = rf_paramGrid)

# getting best model based on grid search
best_model_index = rfmodel.avgMetrics.index(np.max(rfmodel.avgMetrics))
best_model_param = rf_paramGrid[best_model_index]
print(best_model_param)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

{Param(parent='RandomForestClassifier_ef229d5b8961', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 15, Param(parent='RandomForestClassifier_ef229d5b8961', name='numTrees', doc='Number of trees to train (>= 1).'): 200, Param(parent='RandomForestClassifier_ef229d5b8961', name='featureSubsetStrategy', doc="The number of features to consider for splits at each tree node. Supported options: 'auto' (choose automatically for task: If numTrees == 1, set to 'all'. If numTrees > 1 (forest), set to 'sqrt' for classification and to 'onethird' for regression), 'all' (use all features), 'onethird' (use 1/3 of the features), 'sqrt' (use sqrt(number of features)), 'log2' (use log2(number of features)), 'n' (when n is in the range (0, 1.0], use n * number of features. When n is in the range (1, number of features), use n features). default = 'auto'"): 'sqrt'}

In [53]:
# Getting best Evaluation metrics
print("Random Forest - accuracy,precision, recall, f1 :", evaluation(rfresults))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Random Forest - accuracy,precision, recall, f1 : (0.8584441161989168, 0.7470448995970473, 0.403061219348709, 0.5236077004395431)

In [54]:
# Extract feature importance from rfmodel
featImportances = np.array(rfmodel.bestModel.stages[-1].featureImportances)

# List of features
list_feats =  binary_columns + numeric_columns

# Random Forest coefficients
print('Random Forest Features Importances')
print(list_feats)
print(featImportances)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Random Forest Features Importances
['Is_paid', 'Is_Male', 'age_days(t_7)', 'avg_num_songs_per_day(t_7)', 'avg_number_sessions_per_day(t_7)', 'fraction_submit_downgrade(t_7)', 'fraction_error(t_7)', 'fraction_help(t_7)', 'fraction_Roll_Advert(t_7)', 'fraction_Save_Settings(t_7)', 'fraction_Thumbs_Down(t_7)', 'fraction_Submit_Upgrade(t_7)', 'fraction_Add_Friend(t_7)', 'fraction_Add_to_Playlist(t_7)', 'fraction_Thumbs_Up(t_7)']
[0.02499819 0.01450239 0.14396566 0.09779445 0.16161395 0.02620859
 0.03473029 0.05323143 0.07560466 0.03707052 0.07407254 0.05348291
 0.06045111 0.06557656 0.07669676]

**Gradient Boosted Trees**

In [55]:
# GBT with no weights of classes considered

# define model
gb = GBTClassifier()

# define parametes grid
gb_paramGrid = ParamGridBuilder().addGrid(gb.maxDepth,[2,3,5]).addGrid(gb.maxIter,[50,100,200]).build()

# fit model pipeline
gbmodel, gbresults = fit_model(gb, paramGrid = gb_paramGrid)

# getting best model based on grid search
best_model_index = gbmodel.avgMetrics.index(np.max(gbmodel.avgMetrics))
best_model_param = gb_paramGrid[best_model_index]
print(best_model_param)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Exception in thread cell_monitor-55:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/threading.py", line 926, in _bootstrap_inner
    self.run()
  File "/opt/conda/lib/python3.7/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/site-packages/awseditorssparkmonitoringwidget-1.0-py3.7.egg/awseditorssparkmonitoringwidget/cellmonitor.py", line 178, in cell_monitor
    job_binned_stages[job_id][stage_id] = all_stages[stage_id]
KeyError: 3598



{Param(parent='GBTClassifier_2b8b3de6803e', name='maxDepth', doc='Maximum depth of the tree. (>= 0) E.g., depth 0 means 1 leaf node; depth 1 means 1 internal node + 2 leaf nodes.'): 3, Param(parent='GBTClassifier_2b8b3de6803e', name='maxIter', doc='max number of iterations (>= 0).'): 200}

In [58]:
# Getting best Evaluation metrics
print(" Gradient Boosting Trees - accuracy,precision, recall, f1 :", evaluation(gbresults))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

 Gradient Boosting Trees - accuracy,precision, recall, f1 : (0.862383062530773, 0.7429805455079797, 0.4387755046074553, 0.5517194604413324)

In [60]:
# Extract feature importance from gbmodel
featImportances = np.array(gbmodel.bestModel.stages[-1].featureImportances)

# List of features
list_feats =  binary_columns + numeric_columns

# plot logistic regression coefficients
print(list_feats)
print(featImportances)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Is_paid', 'Is_Male', 'age_days(t_7)', 'avg_num_songs_per_day(t_7)', 'avg_number_sessions_per_day(t_7)', 'fraction_submit_downgrade(t_7)', 'fraction_error(t_7)', 'fraction_help(t_7)', 'fraction_Roll_Advert(t_7)', 'fraction_Save_Settings(t_7)', 'fraction_Thumbs_Down(t_7)', 'fraction_Submit_Upgrade(t_7)', 'fraction_Add_Friend(t_7)', 'fraction_Add_to_Playlist(t_7)', 'fraction_Thumbs_Up(t_7)']
[0.03588369 0.00129943 0.1026863  0.16019295 0.12165622 0.05929854
 0.04874183 0.03393979 0.07815162 0.02831667 0.06520274 0.08824231
 0.04440163 0.04937447 0.08261181]

**5.Results Summary**

In [65]:
# Make the results summary for the models considered.
print("Logistic Regression with class weights - Params:","regParam :0.5,threshold :0.5,                        "," Metrics:","F1:","0.50","Accuracy:","0.75")
print("Random Forest                          - Params:","maxDepth:15, numTrees:200, featureSubsetStrategy:sqrt"," Metrics:","F1:","0.52","Accuracy:","0.85")
print("Gradient Boosted Tree                  - Params:","maxDepth :3, maxIter:200                             "," Metrics:","F1:","0.55","Accuracy:","0.86")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Logistic Regression with class weights - Params: regParam :0.5,threshold :0.5,                          Metrics: F1: 0.50 Accuracy: 0.75
Random Forest                          - Params: maxDepth:15, numTrees:200, featureSubsetStrategy:sqrt  Metrics: F1: 0.52 Accuracy: 0.85
Gradient Boosted Tree                  - Params: maxDepth :3, maxIter:200                               Metrics: F1: 0.55 Accuracy: 0.86